In [1]:
import spikeinterface.full as si
import numpy as np

In [2]:
# hack read probe form other fil from spikeglx
file_spikeglx = '/mnt/data/sam/DataSpikeSorting/eduarda_arthur/Rec_1_10_11_2021_g0/'
rec_spikeglx = si.read_spikeglx(file_spikeglx, stream_id='imec0.ap')
probe = rec_spikeglx.get_probe()
# si.plot_probe_map(rec_spikeglx)




In [3]:
probe = probe.copy()
probe.set_device_channel_indices(np.arange(384))

In [4]:
# stabilisation
file = "/crnldata/waking/audrey_hay/NPX/NPX1/VB/Expe_2024-07-22_17-29-40/NP_spikes_2024-07-22T17_29_40.raw"


# vrai enregisstrement
file = "/crnldata/waking/audrey_hay/NPX/NPX1/VB/Expe_2024-07-22_17-55-16/NP_spikes_2024-07-22T17_55_16.raw"

In [10]:
raw_rec = si.read_binary(file, dtype='int16', num_channels=384, sampling_frequency=30_000.)
raw_rec = raw_rec.set_probe(probe)
raw_rec = raw_rec.frame_slice(0, 30_000 * 300)
raw_rec

FrameSliceRecording: 384 channels - 30.0kHz - 1 segments - 9,000,000 samples 
                     300.00s (5.00 minutes) - int16 dtype - 6.44 GiB

In [12]:
sorting = si.run_sorter('kilosort4', raw_rec, verbose=True)
sorting

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 96/96 [00:24<00:00,  3.92it/s]


KiloSortSortingExtractor: 290 units - 1 segments - 30.0kHz

In [13]:
rec = raw_rec.astype('float32')
rec = si.bandpass_filter(rec)
rec = si.common_reference(rec)
rec.get_dtype()

dtype('float32')

In [ ]:
si

In [14]:
si.set_global_job_kwargs(n_jobs=40, progress_bar=True, chunk_duration="1s")

In [16]:
job_kwargs = dict(n_jobs=40, progress_bar=True, chunk_duration="1s")

In [17]:
sorting_analyzer = si.create_sorting_analyzer(sorting, rec, sparse=True)


sorting_analyzer.compute("random_spikes", method="uniform", max_spikes_per_unit=500)
sorting_analyzer.compute("waveforms", **job_kwargs)
sorting_analyzer.compute("templates", **job_kwargs)
sorting_analyzer.compute("noise_levels")
sorting_analyzer.compute("unit_locations", method="monopolar_triangulation")
sorting_analyzer.compute("isi_histograms")
sorting_analyzer.compute("correlograms", window_ms=100, bin_ms=5.)
sorting_analyzer.compute("principal_components", n_components=3, mode='by_channel_global', whiten=True, **job_kwargs)
sorting_analyzer.compute("quality_metrics", metric_names=["snr", "firing_rate"])
sorting_analyzer.compute("template_similarity")
sorting_analyzer.compute("spike_amplitudes", **job_kwargs)


sorting_analyzer.save_as(folder='./sorting_analyzer_demo_K', format='binary_folder')

estimate_sparsity:   0%|          | 0/300 [00:00<?, ?it/s]

compute_waveforms:   0%|          | 0/300 [00:00<?, ?it/s]

Fitting PCA:   0%|          | 0/290 [00:00<?, ?it/s]

Projecting waveforms:   0%|          | 0/290 [00:00<?, ?it/s]

spike_amplitudes:   0%|          | 0/300 [00:00<?, ?it/s]

SortingAnalyzer: 384 channels - 290 units - 1 segments - binary_folder - sparse - has recording
Loaded 11 extensions: random_spikes, waveforms, templates, noise_levels, unit_locations, isi_histograms, correlograms, principal_components, template_similarity, spike_amplitudes, quality_metrics